# T-SNE

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
 
from tensorboard.plugins import projector

2023-03-29 15:30:03.766914: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-29 15:30:03.915944: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-29 15:30:03.916981: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 15:30:05.085075: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load Images

In [4]:

def get_classwise_image_path(csv_file):
    df = pd.read_csv(csv_file)

    ls_normal, ls_bacteria, ls_virus, image_paths_dict = [], [], [], {}
    for path, type in zip(df.file_path, df.type):
        if type == 'normal':
            ls_normal.append(path)
        elif type == 'bacteria':
            ls_bacteria.append(path)
        else:
            ls_virus.append(path)
        
        image_paths_dict.update({'normal':ls_normal, 'bacteria':ls_bacteria, 'virus':ls_virus})
    return image_paths_dict

In [5]:
CSV_FILE = 'dataset.csv'
image_paths_dict = get_classwise_image_path(CSV_FILE)

In [6]:
print(image_paths_dict)

{'normal': ['../data/val/virus/person1319_virus_2276.jpeg', '../data/val/virus/person1318_virus_2274.jpeg', '../data/val/virus/person1316_virus_2271.jpeg', '../data/val/virus/person1317_virus_2273.jpeg', '../data/val/normal/NORMAL2-IM-1431-0001.jpeg', '../data/val/normal/NORMAL2-IM-1440-0001.jpeg', '../data/val/normal/NORMAL2-IM-1442-0001.jpeg', '../data/val/normal/NORMAL2-IM-1436-0001.jpeg', '../data/val/normal/NORMAL2-IM-1430-0001.jpeg', '../data/val/normal/NORMAL2-IM-1437-0001.jpeg', '../data/val/normal/NORMAL2-IM-1427-0001.jpeg', '../data/val/normal/NORMAL2-IM-1438-0001.jpeg', '../data/val/bacteria/person1946_bacteria_4875.jpeg', '../data/val/bacteria/person1947_bacteria_4876.jpeg', '../data/val/bacteria/person1946_bacteria_4874.jpeg', '../data/val/bacteria/person1949_bacteria_4880.jpeg', '../data/test/virus/person1659_virus_2867.jpeg', '../data/test/virus/person1650_virus_2854.jpeg', '../data/test/virus/person1627_virus_2819.jpeg', '../data/test/virus/person10_virus_35.jpeg', '../

## Load Model

In [7]:
IMG_WIDTH, IMG_HEIGHT = (100, 100)

def load_model_and_preprocess_func(input_shape, model_family, model_name):  
     
    # Models will be loaded wth pre-trainied `imagenet` weights.
    model = getattr(tf.keras.applications, model_name)(input_shape=input_shape, 
        weights="imagenet", 
        include_top=False)
     
    preprocess  = getattr(tf.keras.applications, model_family).preprocess_input
    return model, preprocess

def get_feature_extractor(model):
    inputs = model.inputs
    x = model(inputs)
    outputs = tf.keras.layers.GlobalAveragePooling2D()(x)
    feat_ext = tf.keras.Model(inputs=inputs, outputs=outputs, 
        name="feature_extractor")
    return feat_ext

In [8]:
IMAGE_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
MODEL_FAMILY = "resnet"
MODEL_NAME   = "ResNet101"
model, preprocess= load_model_and_preprocess_func(IMAGE_SHAPE, MODEL_FAMILY, MODEL_NAME)
feat_ext_model = get_feature_extractor(model)

In [9]:
# Extract features
def extract_features(input, model, preprocess):
    # Pre-process the input image.
    x = preprocess(input)
    # Generate predictions.
    preds = model.predict(x)
 
    return preds[0]

# Define a method to reshape the image
def load_image_for_inference(image_path, img_shape):
    # Load the image.
    image = tf.io.read_file(image_path)
    # Convert the image from bytes to an image tensor.
    x = tf.image.decode_image(image, channels=img_shape[2])
    # Resize image to the input shape required by the model.
    x = tf.image.resize(x, (img_shape[0], img_shape[1]))
    # Add a dimension for an image batch representation.
    x = tf.expand_dims(x, axis=0)
 
    return x

def load_and_resize_image(img_path, width, height):
    img = Image.open(img_path).resize((width, height))
    return img

def get_images_labels_features(image_paths_dict, feature_extractor, preprocess):
    images = []
    labels = []
    features = []
 
    for cls in image_paths_dict:
        image_paths = image_paths_dict[cls]
        for img_path in image_paths:
            labels.append(cls)
            img = load_and_resize_image(img_path, IMG_WIDTH, IMG_HEIGHT)
            images.append(img)
            img_for_infer = load_image_for_inference(img_path, IMAGE_SHAPE)
            feature = extract_features(img_for_infer, 
                feature_extractor, 
                preprocess)
            features.append(feature)
    return images, labels, features

In [10]:
images, labels, features = get_images_labels_features(image_paths_dict, feat_ext_model, preprocess)

1/1 [==============================] - 0s 113ms/step


2023-03-29 15:31:30.382461: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16070184 exceeds 10% of free system memory.


1/1 [==============================] - 0s 120ms/step


2023-03-29 15:31:30.607651: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16319712 exceeds 10% of free system memory.


1/1 [==============================] - 0s 111ms/step


...

1/1 [==============================] - 0s 113ms/step


2023-03-29 15:31:33.127528: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15415158 exceeds 10% of free system memory.


1/1 [==============================] - 0s 111ms/step


2023-03-29 15:31:33.347076: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15274896 exceeds 10% of free system memory.


1/1 [==============================] - 1s 1s/step


In [ ]:
# Function to Create a Sprite Image
def create_sprite_image(pil_images, save_path):
    # Assuming all images have the same width and height
    img_width, img_height = pil_images[0].size
 
    # create a master square images
    row_coln_count = int(np.ceil(np.sqrt(len(pil_images))))
    master_img_width = img_width * row_coln_count
    master_img_height = img_height * row_coln_count
 
    master_image = Image.new(
        mode = 'RGBA',
        size = (master_img_width, master_img_height),
        color = (0, 0, 0, 0)
    )
 
    for i, img in enumerate(pil_images):
        div, mod = divmod(i, row_coln_count)
        w_loc = img_width * mod
        h_loc = img_height * div
        master_image.paste(img, (w_loc, h_loc))
 
    master_image.convert('RGB').save(save_path, transparency=0)

def write_embedding(log_dir, pil_images, features, labels):
    """Writes embedding data and projector configuration to the logdir."""
    metadata_filename = "metadata.tsv"
    tensor_filename = "features.tsv"
    sprite_image_filename = "sprite.jpg"
 
 
    os.makedirs(log_dir, exist_ok=True)
    with open(os.path.join(log_dir, metadata_filename), "w") as f:
        for label in labels:
            f.write("{}\n".format(label))
    with open(os.path.join(log_dir, tensor_filename), "w") as f:
        for tensor in features:
            f.write("{}\n".format("\t".join(str(x) for x in tensor)))
 
    sprite_image_path = os.path.join(log_dir, sprite_image_filename)
 
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    # Label info.
    embedding.metadata_path = metadata_filename
    # Features info.
    embedding.tensor_path = tensor_filename
    # Image info.
    create_sprite_image(pil_images, sprite_image_path)
    embedding.sprite.image_path = sprite_image_filename
    # Specify the width and height of a single thumbnail.
    img_width, img_height = pil_images[0].size
    embedding.sprite.single_image_dim.extend([img_width, img_height])
    # Create the configuration file.
    projector.visualize_embeddings(log_dir, config)
     
    return

In [ ]:
LOG_DIR = os.path.abspath(os.path.join(os.getcwd(), '../assets', MODEL_NAME))
write_embedding(LOG_DIR, images, features, labels)

## Loading tensorboard

In [ ]:
import os

In [ ]:
MODEL_NAME  = "ResNet101"
LOG_DIR = os.path.abspath(os.path.join(os.getcwd(), '../assets', MODEL_NAME))

In [ ]:
%load_ext tensorboard
# %reload_ext tensorboard
%tensorboard --logdir {LOG_DIR} --host localhost --port 6006 